# Register a dataframe as a table

In [ ]:
df = sqlContext.read.json('data/people.json')
df.registerTempTable('people')

sqlContext.sql('show tables').show()

# Working with SQL

In [ ]:
# select
sqlContext.sql('select * from people').show()

In [ ]:
# filter
sqlContext.sql('select * from people where name="Andy"').show()

In [ ]:
# groupby
sqlContext.sql('select name, count(*) as number from people group by name').show()

# SQL Join

In [ ]:
from pyspark.sql import Row
df1 = sc.parallelize(
    [Row(name='Alice', age=5),
     Row(name='Tom', age=3),
     Row(name='Jerry', age=10)]).toDF()

df1.registerTempTable('df1')


df2 = sc.parallelize(
    [Row(name='Alice', height=80),
     Row(name='Tom', height=75),
     Row(name='John', height=60)]).toDF()

df2.registerTempTable('df2')

In [ ]:
# inner join
sqlContext.sql('select df1.name, age, height from df1 join df2 on df1.name=df2.name').show()

In [ ]:
# left join
sqlContext.sql('select df1.name, age, height from df1 left join df2 on df1.name=df2.name').show()

In [ ]:
# outer join
sqlContext.sql('select df1.name, age, height from df1 full outer join df2 on df1.name=df2.name').show()

# Using Common Table Expressions

# Embeded SQL

In [ ]:
sql = """
select * from df1 join 
(
    select name, count(*) as number from df2 group by name
) as my_count
on 
    df1.name = my_count.name
"""
sqlContext.sql(sql).show()

## "With" statement

In [ ]:
sql = """
 with my_count as (
 select name, count(*) as number 
 from df2 where df2.height > 75 group by name )
select * from df1 join my_count
on
    df1.name = my_count.name
"""
sqlContext.sql(sql).show()

## "Union" Statement

In [ ]:
sql = """
select * from df1 where df1.name='Alice'
union
select * from df1 where df1.age < 5
"""
sqlContext.sql(sql).show()

# Save Dataframe as Parquet file

In [ ]:
sql = """
select * from df1 where df1.name='Alice'
union
select * from df1 where df1.age < 5
"""
r = sqlContext.sql(sql)
r.write.format('parquet').saveAsTable('result')

In [ ]:
sqlContext.sql('show tables').toPandas()